# Building LTV target

We can have several LTV models, depending on when we want to apply it. For instance, if we want a model for recently acquired clients, to direct hunting for example, we are bound to registration information or open market information, while a model for farming can use transactions and client's behavior as variables. 
The same is valid for the target, as we can compute the lifetime value for the client since the beginning of the relationship or from a determined point forward. All these definitions depend on business objectives and use cases of the models.

Since this is a study on the subject, we are going to build two views:
- Considering all revenue from a client since the beginning, limiting the prediction variables to the registration and first transaction, as for a new costumer.
- Considering all revenue generated from one specific point forward, as for a current user.

We are also checking for one-and-done cases to evaluate what types of strategies we can apply to increase revenue.

In [1]:
import pandas as pd

# Data

In [2]:
orders = pd.read_csv('../data/bronze/olist_orders_dataset.csv')
order_items = pd.read_csv('../data/bronze/olist_order_items_dataset.csv')
order_payments = pd.read_csv('../data/bronze/olist_order_payments_dataset.csv')

# Understanding order payments

For LTV, we will consider only orders paid by the costumer. Vouchers and other types os incentive are not considered.

In [3]:
order_payments[order_payments['order_id'].duplicated()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
1456,683bf306149bb869980b68d48a1bd6ab,1,credit_card,1,8.58
2324,e6a66a8350bb88497954d37688ab123e,2,voucher,1,10.51
2393,8e5148bee82a7e42c5f9ba76161dc51a,1,credit_card,1,0.67
2414,816ccd9d21435796e8ffa9802b2a782f,1,credit_card,1,5.65
2497,2cbcb371aee438c59b722a21d83597e0,2,voucher,1,7.80
...,...,...,...,...,...
103778,fd86c80924b4be8fb7f58c4ecc680dae,1,credit_card,1,76.10
103817,6d4616de4341417e17978fe57aec1c46,1,credit_card,1,19.18
103860,31bc09fdbd701a7a4f9b55b5955b8687,6,voucher,1,77.99
103869,c9b01bef18eb84888f0fd071b8413b38,1,credit_card,6,238.16


In [4]:
order_payments[order_payments['order_id']=='c9b01bef18eb84888f0fd071b8413b38']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
10326,c9b01bef18eb84888f0fd071b8413b38,2,voucher,1,128.44
103869,c9b01bef18eb84888f0fd071b8413b38,1,credit_card,6,238.16


In [5]:
order_payments['payment_type'].value_counts()

payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

In [6]:
# The methods considered valid for our purposes are: credit_card, boleto (Bank slip) and debit_card.
valid_methods = ['credit_card', 'boleto', 'debit_card']

In [7]:
# since there can be multiple payment method for ear order, now we will filter the valid ones and aggregate their values.
valid_order_payments = order_payments[order_payments['payment_type'].isin(valid_methods)].copy()
valid_order_payments = valid_order_payments.groupby('order_id').agg({'payment_value':['sum']}).reset_index()
valid_order_payments.columns = [x[0] for x in valid_order_payments.columns]

In [8]:
valid_order_payments

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04
...,...,...
97811,fffc94f6ce00a00581880bf54a75a037,343.40
97812,fffcd46ef2263f404302a634eb57f7eb,386.53
97813,fffce4705a9662cd70adb13d4a31832d,116.85
97814,fffe18544ffabc95dfada21779c9644f,64.71


In [9]:
valid_order_payments['order_id'].duplicated().sum()

0

# Merging clients and order dates

We can use the order_purchse_timestamp from the orders dataset to get costumers_id and order dates.
To get customer unique id we need customers dataset

In [10]:
customers = pd.read_csv('../data/bronze/olist_customers_dataset.csv')

In [11]:
orders['order_id'].duplicated().sum() #  there are no duplicated orders here, as expected.

0

In [12]:
orders_filtered = orders.merge(valid_order_payments,on='order_id',how='inner') # we do an inner join since we are interested only in the orders paid with the valid methods.
orders_filtered = orders_filtered.merge(customers[['customer_id','customer_unique_id']],on='customer_id',how='inner') # we use inner so we only get traceable clients.

In [13]:
orders_filtered['order_status'].value_counts() # we algo only want fineshed orders, so we now the client actually paid the price.

order_status
delivered      94979
shipped         1079
unavailable      599
canceled         546
invoiced         312
processing       294
created            5
approved           2
Name: count, dtype: int64

In [14]:
valid_status = ['delivered']

In [15]:
orders_filtered = orders_filtered[orders_filtered['order_status'].isin(valid_status)].copy()

# Build target

As mentioned before, we will build the target in two views dependending on how long the relationship with the client is.

In [16]:
orders_filtered = orders_filtered.sort_values(['customer_unique_id','order_purchase_timestamp'])

In [17]:
# we need to number each order from each client so we can filter the first one later. We will also use this number to analyse possible one and done and churn.
orders_filtered['client_order_number'] = orders_filtered.sort_values(['customer_unique_id','order_purchase_timestamp']).groupby(['customer_unique_id']).cumcount()+1

In [18]:
orders_filtered['client_order_number'].value_counts(1) 
# Seems like 97% of all orders are made by clients that never returns. That means that an one and done model can also be applied to this dataset.
# A complete solution would include evaluating each costumer after their first order in terms of one-and-done and them in LTV.
# for now, we won't filter these clients out.

client_order_number
1     0.968298
2     0.028501
3     0.002253
4     0.000484
5     0.000200
6     0.000105
7     0.000053
8     0.000021
9     0.000021
10    0.000011
11    0.000011
12    0.000011
13    0.000011
14    0.000011
15    0.000011
Name: proportion, dtype: float64

## Time between transactions
We need to determine a window to consider LTV, so we need to take a look at recency and frequency. We also apply a floor to the purchase date, since we are not interested is such granularity.

In [19]:
orders_filtered['order_purchase_timestamp'] = orders_filtered['order_purchase_timestamp'].apply(pd.to_datetime)

In [20]:
orders_filtered['trns_ref'] =  orders_filtered['order_purchase_timestamp'].dt.floor('d')

In [21]:
orders_filtered['time_lst_order'] = orders_filtered.groupby(['customer_unique_id'])['trns_ref'].diff(periods=1).dt.days

In [22]:
orders_filtered['time_nxt_order'] = orders_filtered.groupby(['customer_unique_id'])['trns_ref'].diff(periods=-1).dt.days.abs()

In [23]:
orders_filtered['time_nxt_order'].value_counts()

time_nxt_order
0.0      871
1.0       48
2.0       40
7.0       34
6.0       32
        ... 
328.0      1
442.0      1
356.0      1
291.0      1
322.0      1
Name: count, Length: 405, dtype: int64

In [24]:
orders_filtered['time_lst_order'].value_counts()

time_lst_order
0.0      871
1.0       48
2.0       40
7.0       34
6.0       32
        ... 
328.0      1
442.0      1
356.0      1
291.0      1
322.0      1
Name: count, Length: 405, dtype: int64

In [25]:
orders_filtered['time_lst_order'].describe() 
# Only 3011 repurchases
# Of those, 75% will buy again within the next 4 months (121 days) after the last purchase.

count    3011.000000
mean       78.796081
std       107.033942
min         0.000000
25%         0.000000
50%        29.000000
75%       121.000000
max       609.000000
Name: time_lst_order, dtype: float64

## Clients lifetime

In [26]:
lifetime_df = orders_filtered.groupby(['customer_unique_id'])['trns_ref'].agg(['min','max']).diff(axis=1).rename(columns={'max':'lifetime'}).reset_index().drop(columns='min')
lifetime_df['lifetime'] = lifetime_df['lifetime'].dt.days

In [27]:
lifetime_df['lifetime'].describe() # lets filter out one and done cases to see the behavior

count    91968.000000
mean         2.579756
std         24.675138
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        633.000000
Name: lifetime, dtype: float64

In [28]:
lifetime_df[lifetime_df['lifetime']>0]['lifetime'].describe() # 75% of clients suvive for around 190 days
# combining this with results from before we can see that clients that make a second interaction, do it whithin the next 6 months

count    1933.000000
mean      122.739265
std       119.277515
min         1.000000
25%        25.000000
50%        81.000000
75%       187.000000
max       633.000000
Name: lifetime, dtype: float64

From the above results we can select a window for LTV of 6 months.
And since most clients only make a second buy and not more than that, we will change our views of the problem to:
- one and done modelling
- LTV considering a window of 6 months from the first purchase

## One and Done

The first target we are going to build is the one-and-done, which check if a client will do a second purchase.
In this case, we will consider a window twice as long as the one for LTV, just so we don't have samples in which a client comes back after too long and is treated as good.
These definitions are usually based on business decisions, but for now they seem resonable.

In [29]:
orders_filtered_grouped = orders_filtered.groupby(['customer_unique_id','trns_ref']).agg({'payment_value':'sum','client_order_number':'min'}).reset_index(drop=False)

In [30]:
df_oad = orders_filtered_grouped.copy()

In [31]:
df_oad['time_nxt_order'] = df_oad.groupby(['customer_unique_id'])['trns_ref'].diff(periods=-1).dt.days.abs() # we recalculate the recency now base on the day of the next transaction.

In [32]:
df_oad = df_oad[df_oad['client_order_number']==1].copy()

In [33]:
mask_oad = (df_oad['time_nxt_order'].isna() ) | (df_oad['time_nxt_order']>180 )

In [34]:
df_oad['one_and_done'] = 0

In [35]:
df_oad.loc[mask_oad,'one_and_done']=1

In [36]:
df_oad.rename(columns={'trns_ref':'ref'},inplace=True)

In [37]:
df_oad['one_and_done'].value_counts(1) # only 1.59% of all clients are not one and done

one_and_done
1    0.984038
0    0.015962
Name: proportion, dtype: float64

# LTV
Similar to one and done, we are useing the date of the first purchase as our referece. In production, this models would be used at this same time.


In [38]:
df_ltv =orders_filtered_grouped.copy()

In [39]:
df_ltv['ref'] = df_ltv.groupby('customer_unique_id')['trns_ref'].transform('min')
date_filter = (df_ltv['trns_ref']>df_ltv['ref']) & (df_ltv['trns_ref']<=(df_ltv['ref']+ pd.DateOffset(days=180))) 
df_ltv = df_ltv[date_filter].groupby(['customer_unique_id','ref'])['payment_value'].sum().to_frame().reset_index(drop=False).rename(columns = {'payment_value':'LTV'})

In [40]:
df_ltv[df_ltv['LTV']>0].sort_values('LTV')

,customer_unique_id,ref,LTV
1212,d02cbb85434c84acabb8e81bbf064d3e,2017-09-16,0.33
624,6e1550b8966f327f29bc06b18972dbec,2017-05-20,0.52
494,55fbb64f60861980e74c4f136f304297,2018-04-27,1.97
245,2a7f837de6a1999e457b8e3c5619156f,2018-03-27,2.14
500,5709900c6ded8af6dfb28b043943368d,2017-12-16,2.50
...,...,...,...
334,397b44d5bb99eabf54ea9c2b41ebb905,2018-01-11,1429.92
4,012a218df8995d3ec3bb221828360c86,2018-05-07,1435.97
769,86df00dc5fd68f4dd5d5945ca19f3ed6,2017-06-08,1535.88
934,a1044dd75b74fbc485b040575a14acf0,2017-12-12,1650.18


In [41]:
df_ltv.shape

(1468, 3)

### Check values

In [42]:
df_ltv[df_ltv['customer_unique_id'].isin(['011b4adcd54683b480c4d841250a987f','c8460e4251689ba205045f3ea17884a1'])].sort_values('customer_unique_id')

,customer_unique_id,ref,LTV
3,011b4adcd54683b480c4d841250a987f,2017-08-22,149.88
1161,c8460e4251689ba205045f3ea17884a1,2018-08-07,2405.28


In [43]:
orders_filtered[orders_filtered['customer_unique_id'].isin(['011b4adcd54683b480c4d841250a987f','c8460e4251689ba205045f3ea17884a1'])].sort_values('customer_unique_id')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_value,customer_unique_id,client_order_number,trns_ref,time_lst_order,time_nxt_order
23149,f05a68aaa0d8f89e758c7134d53fa22a,7e16c78f08de9b2ec5b9eff657934019,delivered,2017-08-22 12:51:29,2017-08-22 13:08:00,2017-08-23 18:53:21,2017-09-04 20:54:01,2017-09-22 00:00:00,86.42,011b4adcd54683b480c4d841250a987f,1,2017-08-22,NaN,177.0
12289,93b35affb86435b71ca01a6dd1eab2b1,3db635076c27c1d6a8618bc5a6e70ad2,delivered,2018-02-15 11:40:57,2018-02-15 12:40:28,2018-02-16 00:07:53,2018-02-27 19:23:00,2018-03-13 00:00:00,149.88,011b4adcd54683b480c4d841250a987f,2,2018-02-15,177.0,NaN
36526,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,delivered,2018-08-07 09:03:02,2018-08-08 09:05:09,2018-08-08 15:01:00,2018-08-15 19:28:29,2018-08-24 00:00:00,1202.64,c8460e4251689ba205045f3ea17884a1,1,2018-08-07,NaN,0.0
72387,3e073b27114fe955e184db124d34a566,6ad78e330d6e0f3dfc471115f17371ea,delivered,2018-08-07 16:35:35,2018-08-08 14:10:16,2018-08-13 15:35:00,2018-08-21 17:18:55,2018-09-19 00:00:00,1047.99,c8460e4251689ba205045f3ea17884a1,2,2018-08-07,0.0,1.0
69004,8e8f8ef29c78c7b9c98cac70dd331ce2,9e478f18132211e1d6cf2637d12cc559,delivered,2018-08-08 14:24:24,2018-08-09 08:50:25,2018-08-14 18:00:00,2018-08-21 15:32:26,2018-08-27 00:00:00,1202.64,c8460e4251689ba205045f3ea17884a1,3,2018-08-08,1.0,0.0
42985,03aba68b07658f28f29612641f08d4ba,a7ab31829dc9a10f37e82b1e1afd26b6,delivered,2018-08-08 14:27:15,2018-08-09 08:50:17,2018-08-14 18:00:00,2018-08-21 15:33:32,2018-08-27 00:00:00,1202.64,c8460e4251689ba205045f3ea17884a1,4,2018-08-08,0.0,NaN


In [44]:
# the values are correct, but we can see that there are clients that made several purchases in a very short period and never came back. 
# It will be a business decision whether this behavior is good or bad, but since we have few samples, we will keep them.

# Final target dataframe

In [45]:
df_targets = df_oad[['customer_unique_id','ref','one_and_done']].copy()

In [46]:
df_targets = df_targets.merge(df_ltv,on=['customer_unique_id','ref'],how='left')

## Sanity check
All one and done clients must have 0 LTV

In [47]:
df_targets['one_and_done'].value_counts(1,dropna=False)

one_and_done
1    0.984038
0    0.015962
Name: proportion, dtype: float64

In [48]:
(df_targets['LTV']>0).value_counts(1,dropna=False)

LTV
False    0.984038
True     0.015962
Name: proportion, dtype: float64

In [49]:
df_targets[(df_targets['LTV']==0) & (df_targets['one_and_done']==0)] # These are cases when 2 purchases accour at same time, so it makes sense the ltv is 0.

,customer_unique_id,ref,one_and_done,LTV


In [50]:
df_targets[(df_targets['LTV']>0) & (df_targets['one_and_done']!=0)] # These are cases when 2 purchases accour at same time, so it makes sense the ltv is 0.

,customer_unique_id,ref,one_and_done,LTV


# Saving data

In [51]:
df_targets.sort_values('LTV')

,customer_unique_id,ref,one_and_done,LTV
74837,d02cbb85434c84acabb8e81bbf064d3e,2017-09-16,0,0.33
39653,6e1550b8966f327f29bc06b18972dbec,2017-05-20,0,0.52
30930,55fbb64f60861980e74c4f136f304297,2018-04-27,0,1.97
15272,2a7f837de6a1999e457b8e3c5619156f,2018-03-27,0,2.14
31307,5709900c6ded8af6dfb28b043943368d,2017-12-16,0,2.50
...,...,...,...,...
91963,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08,1,NaN
91964,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10,1,NaN
91965,ffff371b4d645b6ecea244b27531430a,2017-02-07,1,NaN
91966,ffff5962728ec6157033ef9805bacc48,2018-05-02,1,NaN


In [52]:
df_targets.to_parquet('../data/silver/df_targets.parquet')